In [1]:
#@title Imports { form-width: "20%" }
# Some imports that might be useful one day
import numpy as np
import urllib.request
import re
import collections
import itertools
import heapq

def get_input(day):
  return urllib.request.urlopen(f'https://raw.githubusercontent.com/SantoSimone'
                                f'/Advent-of-Code/master/2020/input_files/input'
                                f'{day}.txt').read().decode('utf-8')

def get_input_as_lines(day):
  return get_input(day).split('\n')[:-1]

def parse_ints(text):
  return [int(x) for x in re.findall(r'\d+', text)]

# Day 1

In [ ]:
#@title Part 1 { form-width: "20%" }

def d1p1():
  expenses = parse_ints(get_input(day))
  for val1, val2 in itertools.combinations(expenses, r=2):
    if val1 + val2 == 2020: return val1, val2 

# 'And so it begins' 
# (for non-nerds: this is a LOTR reference timing 2:46:41 enjoy!)
day = 1
val1, val2 = d1p1()
print(f'The two entries that sum to 2020 are: {val1} and {val2}. Their product'
      f' is {val1*val2}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def d1p2():
  expenses = parse_ints(get_input(day))
  for val1, val2, val3 in itertools.combinations(expenses, r=3):
    if val1 + val2 + val3 == 2020: return val1, val2, val3

val1, val2, val3 = d1p2()
print(f'The three entries that sum to 2020 are: {val1}, {val2} and {val3}. '
      f'Their product is {val1*val2*val3}')

# Day 2

In [ ]:
#@title Part 1 { form-width: "20%" }

def d2p1(inputs):
  def check_policy(line):
    least, most, ch, pw = re.match(r'(\d+)-(\d+) (\w): (\w+)', line).groups()
    return int(least) <= collections.Counter(pw)[ch] <= int(most)

  return sum([check_policy(line) for line in inputs])
  
# It's day 2
day = 2
valid = d2p1(get_input_as_lines(day))
print(f'# of valid passwords: {valid}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Policy has changed, so we will change our policy func
def d2p2(inputs):
  def check_policy(line):
    least, most, ch, pw = re.match(r'(\d+)-(\d+) (\w): (\w+)', line).groups()
    return (pw[int(least) - 1] == ch) ^ (pw[int(most) - 1] == ch)

  return sum([check_policy(line) for line in inputs])
  

valid = d2p2(get_input_as_lines(day))
print(f'# of valid passwords: {valid}')

# Day 3

In [ ]:
#@title Part 1 { form-width: "20%" }

def d3p1(lines, start, slope):
    def generator(start, slope, width):
        i = 0
        while True:
            yield start[0] + (slope[0] * i) % width, start[1] + slope[1] * i
            i += 1

    def create_map(lines):
        return np.array([
            [x == '#' for x in line]  # columns
            for line in lines  # rows
        ], dtype=np.int32)  # ones and zeros are always nice

    tree_map = create_map(lines)
    height, width = tree_map.shape
    sum = 0
    for c, r in generator(start, slope, width):
        if r >= height: return sum
        sum += tree_map[r, c]

# It's day 3
day = 3
start = (0, 0)
slope = (3, 1) 
trees = d3p1(get_input_as_lines(day), start, slope)
print(f'# of trees with first policy: {trees}')

# of trees with first policy: 167


In [ ]:
#@title Part 2 { form-width: "20%" }

# Not an hard request, we simply need to call previous func 5 times
def d3p2(lines, start, slopes):
  return np.prod([d3p1(lines, start, slope) for slope in slopes])
  
slopes = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
trees = d3p2(get_input_as_lines(day), start, slopes)
print(f'Multiplied values of trees encountered with 5 policies: {trees}')

Multiplied values of trees encountered with 5 policies: 736527114


# Day 4

In [ ]:
#@title Part 1 { form-width: "20%" }
def parse_input(lines):
    ret = []
    passport = {}
    for line in lines:
        if line == '':
            ret.append(passport)
            passport = {}
            continue
        parts = line.split()
        for p in parts:
            k, v = p.split(':')
            passport[k] = v
    ret.append(passport)

    return ret

def check_passport(passport, required_fields):
  return all(field in passport for field in required_fields)

def d4p1(lines):
  passports = parse_input(lines)
  return np.sum([check_passport(passport, required_fields) 
                   for passport in passports])

# It's day 4
day = 4
lines = get_input_as_lines(day)
required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
optional_fields = ['cid']
s = d4p1(lines)
print(f'Valid passports: {s}')

Valid passports: 239


In [ ]:
#@title Part 2 { form-width: "20%" }

# Now we want also data validation
def data_validation(passport):
  valid = True
  try:
      num, unit = re.match(r'(\d+)(\w+)', passport['hgt']).groups()
      if unit == 'cm':
          valid = (valid and (150 <= int(num) <= 193))
      elif unit == 'in':
          valid = (valid and (59 <= int(num) <= 76))
      else:
          return False

      return valid and (1920 <= int(passport['byr']) <= 2002) \
              and (2010 <= int(passport['iyr']) <= 2020) \
              and (2020 <= int(passport['eyr']) <= 2030) \
              and passport['hcl'][0] == '#' and len(passport['hcl']) == 7 \
              and all('0' <= c <= '9' or 'a' <= c <= 'f' for c in passport['hcl'][1:]) \
              and passport['ecl'] in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'] \
              and int(passport['pid']) and len(passport['pid']) == 9
  except:
      return False

def d4p2(lines):
  passports = parse_input(lines)
  return np.sum([check_passport(passport, required_fields)
                 and data_validation(passport)
                 for passport in passports])
  

s = d4p2(lines)
print(f'Valid passports with valid data: {s}')

Valid passports with valid data: 188


# Day 5

In [46]:
#@title Part 1 { form-width: "20%" }

def binary_space(chars, upper, lower, low_char, high_char):
  for c in chars:
    lower += (upper - lower + 1) / 2 if c == low_char else 0
    upper -= (upper - lower + 1) / 2 if c == high_char else 0
  return upper

def d5p1(lines):
  return [
          int(binary_space(seat[:7], 127, 0, 'B', 'F') * 8 
          + binary_space(seat[7:], 7, 0, 'R', 'L'))
          for seat in lines
          ]

# It's day 5
day = 5
lines = get_input_as_lines(day)
highest = max(d5p1(lines))
print(f'Highest seat ID: {highest}')

Highest seat ID: 922


In [47]:
#@title Part 2 { form-width: "20%" }

def check_pre_post(id, ids):
  return id if id + 1 in ids and id - 1 in ids and id not in ids else False

def d5p2(lines):
  ids = d5p1(lines)
  return np.sum([check_pre_post(id, ids) for id in range(min(ids), max(ids))])

my_id = d5p2(lines)
print(f'My ID is: {my_id}')

My ID is: 747
